In [ ]:
import pandas as pd
import numpy as np
import random
import pickle
import itertools
import logging
import re
import time

import matplotlib.pyplot as plt
import seaborn as sns

from konlpy.tag import Okt
#from gensim.models import Word2Vec, KeyedVectors
from collections import Counter
from tqdm import tqdm
from sklearn import preprocessing

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_excel('./data/all_10_random_12000_262_train.xlsx')
val = pd.read_excel('./data/all_10_random_12000_262_val.xlsx')

In [ ]:
train

In [ ]:
val

In [ ]:
train_abst = train['abstract']
val_abst = val['abstract']

In [ ]:
train_sent = list(train_abst)
val_sent = list(val_abst)

In [ ]:
len(train_sent)

In [ ]:
len(val_sent)

In [ ]:
sentences = train_sent + val_sent

In [ ]:
#sentences

In [ ]:
len(sentences)

In [ ]:
type(sentences)

# Preprocess

In [ ]:
hangul = re.compile('[^ ㄱ-ㅣ가-힣0-9]+')
preprocessed_data = []

for sent in tqdm(sentences):
    result = hangul.sub('', sent)
    preprocessed_data.append(result)

In [ ]:
len(preprocessed_data)

In [ ]:
type(preprocessed_data)

# Tokenize

In [ ]:
okt = Okt()
tokenized_data = []
stopwords = ['을', '를', '이', '가', '은', '는', '의']

for sentence in tqdm(preprocessed_data):
    token = okt.morphs(sentence) # 토큰화
    token = [t for t in token if not t in stopwords]
    tokenized_data.append(token)

In [ ]:
tokenized_data[:5]

In [ ]:
len(tokenized_data)

In [ ]:
type(tokenized_data)

In [ ]:
print('초록의 최대 길이 :',max(len(l) for l in tokenized_data))
print('초록의 평균 길이 :',sum(map(len, tokenized_data))/len(tokenized_data))
plt.hist([len(s) for s in tokenized_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
# save
with open('./data/embedding/soy_tokenized_data_all_random_262.pickle', 'wb') as f:
    pickle.dump(tokenized_data, f, pickle.HIGHEST_PROTOCOL)

# Train

In [ ]:
from gensim.models import Word2Vec, KeyedVectors

In [ ]:
# load
with open('./data/embedding/soy_tokenized_data_all_random_262.pickle', 'rb') as f:
    tokenized_data = pickle.load(f)

In [ ]:
import multiprocessing

In [ ]:
model = Word2Vec(sentences = tokenized_data, size = 100, window = 5, min_count = 1, iter=300,  workers = multiprocessing.cpu_count(), sg = 1)

In [ ]:
model.save('word2vec_okt_all_random_262.model')

# Inference

In [ ]:
from gensim.models import Word2Vec, KeyedVectors

In [ ]:
model = Word2Vec.load('./data/embedding/word2vec_okt_all_random_262.model')

In [ ]:
word_vectors = model.wv
word_vectors

In [ ]:
word_vectors['딥러닝']

In [ ]:
vocabs = word_vectors.vocab.keys()
vocabs

In [ ]:
word_vectors_list = [word_vectors[v] for v in vocabs]
word_vectors_list

In [ ]:
model_result1 = model.wv.most_similar("인공지능")
print(model_result1)

In [ ]:
word_vectors.similarity(w1 = '딥러닝', w2 = '인공지능')